The primary objective of this project is to leverage machine learning for predicting the revenues of ABB Electrification's business area. If the dataset proves suitable, an additional aim is to extend this methodology to ABB's industry counterparts.

To achieve these goals, we will conduct an in-depth analysis of the following datasets:
- Revenues CSV file for ABB and its peers (current as of Q3 2023)
- Macroeconomic indicators from the World Economic Forum (WEF) CSV file (reflecting data as of October 2023)

In [280]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from math import sqrt
from scipy.interpolate import interp1d
from sklearn.svm import SVR
import xgboost as xgb


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

ModuleNotFoundError: No module named 'xgboost'

# Revenues file - Data modelling

In [ ]:
# Loading the dataset with Company Revenue and Profitability
df = pd.read_csv("Financials v2 - Copy.csv")

df

In [ ]:
# Unpivot the DataFrame, change column types and change the 'CIQ Formula column' values
df = pd.melt(df, id_vars=['Ticker', 'Company name', 'Segment', 'Segment Number', 'CIQ Formula', 'Currency'], var_name='Date', value_name='Value')
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df['Value'] = pd.to_numeric(df['Value'], errors='coerce')
df['CIQ Formula'] = df['CIQ Formula'].replace({'IQ_BUS_SEG_REV': 'Revenue', 'IQ_BUS_SEG_OPER_INC_ABS': 'Profitability', 'IQ_BUS_SEG_EBITDA_ABS': 'Profitability'})
df = df.sort_values(by=['Ticker', 'Date'])

df
# Data types sanity-check
print(df.dtypes)

In [ ]:
# Check how many missing values are in 'Value' column. Focus on Revenues.
zero_table = df[(df['Value'] == 0) & (df['CIQ Formula'] == 'Revenue')]
zero_table

Now we have confirmed that each column has been assigned the correct data type, but we've identified numerous missing values that require attention. Imputing the mean value may not be feasible due to the high prevalence of missing data. Let's explore whether Linear Regression would be a suitable approach.

In [ ]:
# Extract the year and month from the 'Date' column
df['YearMonth'] = df['Date'].dt.to_period('M')

# Map Year-Month to a numerical representation
df['YearMonthNumeric'] = df['YearMonth'].apply(lambda x: x.year * 12 + x.month)

# Select rows with 'Value' == 0 and 'Value' != 0
df_missing = df[df['Value'] == 0]
df_not_missing = df[df['Value'] != 0].copy()  # Make a copy to avoid the SettingWithCopyWarning

# Create a new column combining 'Ticker', 'Currency', 'CIQ Formula', and 'Segment' for unique combinations
df_not_missing['UniqueCombination'] = (
    df_not_missing['Ticker'] + '_' + df_not_missing['Currency'] + '_' +
    df_not_missing['CIQ Formula'] + '_' + df_not_missing['Segment']
)

# Initialize an empty DataFrame to store the results
result_df = pd.DataFrame()

# Iterate over unique combinations
for combination, group in df_not_missing.groupby('UniqueCombination'):
    # Check if there are any samples for training
    if len(group) > 0:
        # Prepare data for regression
        X_train = group[['YearMonthNumeric']].values.reshape(-1, 1)
        y_train = group['Value'].values

        # Filter the missing values based on the current combination
        missing_filter = (
            (df_missing['Ticker'] + '_' + df_missing['Currency'] + '_' +
             df_missing['CIQ Formula'] + '_' + df_missing['Segment']) == combination
        )
        indices = df_missing.loc[missing_filter].index

        # Check if there are any missing values for the current combination
        if len(indices) > 0:
            # Initialize the linear regression model
            model = LinearRegression()

            # Train the model
            model.fit(X_train, y_train)

            # Predict missing values
            predicted_values = model.predict(df_missing.loc[missing_filter, ['YearMonthNumeric']].values.reshape(-1, 1))

            # Update DataFrame with predicted values using indices
            df.loc[indices, 'Value'] = predicted_values
    else:
        print(f"Not enough samples for {combination}, skipping...")

df

In [ ]:
# Check if there are still some missing data in dataset. Lets use ABB, Electrification example
abb_data = df[(df['Company name'] == 'ABB') & (df['CIQ Formula'] == 'Revenue') & (df['Segment'] == 'Electrification')]
abb_data

# Revenues file - Drawing a chart for ABB Electrification

In [ ]:
# Create the plot chart
abb_data = abb_data.sort_values(by='Date')
plt.figure(figsize=(10, 6))
plt.plot(abb_data['Date'], abb_data['Value'], linestyle='-', color='r')
plt.title('Evolution of ABB EL revenue, USD')
plt.xlabel('Date')
plt.ylabel('Revenue (USD)')
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()

plt.show()

Linear regression might be not the best solution here.
For the period from 2010 to 2015 the line it "too flat". Lets see it similar situtation is occuring in other company - for example Siemens.

In [ ]:
# Filter for Siemens, Smart Infrastructure
siemens_data = df[(df['Company name'] == 'Siemens') & (df['CIQ Formula'] == 'Revenue') & (df['Segment'] == 'Industrial Businesses (IB) - Smart Infrastructure') & (df['Currency'] == 'USD')]
siemens_data

In [ ]:
# Create the plot
siemens_data = siemens_data.sort_values(by='Date')
plt.figure(figsize=(10, 6))
plt.plot(siemens_data['Date'], siemens_data['Value'], linestyle='-', color='c')
plt.title('Evolution of Siemens SI revenue, USD')
plt.xlabel('Date')
plt.ylabel('Revenue (USD)')
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()

plt.show()

The situation is even worse. Here, data is missing until 2018.

# Revenues file - Checking other companies

In [ ]:
# Group by'Company Name' i 'Date' and summarize Revenues in USD
revenue_data = df[df['CIQ Formula'] == 'Revenue']
summed_data = revenue_data.groupby(['Company name', 'Date', 'Currency'], as_index=False)['Value'].sum()
summed_data_usd = summed_data[summed_data['Currency'] == 'USD']

# Create the plot
plt.figure(figsize=(12, 8))

# Go through all companies
for company in summed_data_usd['Company name'].unique():
    company_data = summed_data_usd[summed_data_usd['Company name'] == company]
    plt.plot(company_data['Date'], company_data['Value'], label=company, marker='o', linestyle='-')

plt.title('Sum of Revenue for Each Company in USD')
plt.xlabel('Date')
plt.ylabel('Sum of Revenue (USD)')
plt.xticks(rotation=45)
plt.legend(bbox_to_anchor=(1, 1), loc='upper left')  # Umieść legendę poza wykresem
plt.grid(True)
plt.tight_layout()

plt.show()

In summary, it's crucial to note that a comprehensive analysis is hindered by the absence of reliable pre-2018 data for certain companies, such as Wesco and Siemens. Therefore, to align with the primary objective of developing a ML model tailored to ABB Electrification, our focus will be exclusively on that specific dataset.

# Revenues file - Applying 2028 Revenues prediction for ABB, Electrification

Let's see how it would look like with pre 2015 data incorporated.

In [ ]:
# Prepare the Revenue data for ABB, Electrification
revenue_data = df[(df['CIQ Formula'] == 'Revenue') & (df['Segment'] == 'Electrification')]
summed_data = revenue_data.groupby(['Company name', 'Date', 'Currency'], as_index=False)['Value'].sum()
abb_data = summed_data[summed_data['Company name'] == 'ABB']
end_date = '2028-12-31'

def linear_regression_and_metrics(data, end_date='2028-12-31', test_size=0.2, random_state=42):
    # Linear regression model
    X = (data['Date'] - data['Date'].min()).dt.days.values.reshape(-1, 1)
    y = data['Value']

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

    # Train the model on the training set
    model = LinearRegression()
    model.fit(X_train, y_train)

    # Predict revenues for the entire dataset
    y_pred = model.predict(X)

    # Generate dates up to the specified end date
    future_dates = pd.date_range(start=data['Date'].min(), end=end_date, freq='M')
    future_days = (future_dates - data['Date'].min()).days.values.reshape(-1, 1)

    # Predict revenues for future dates
    future_predictions = model.predict(future_days)

    # Create the plot
    plt.figure(figsize=(12, 6))
    plt.scatter(data['Date'], data['Value'], label='Actual Data', color='red')
    plt.plot(data['Date'], y_pred, label='Linear Regression', color='blue')
    plt.plot(future_dates, future_predictions, label='Predicted Data', linestyle='dashed', color='green')
    plt.xlabel('Date')
    plt.ylabel('Revenue (USD)')
    plt.title('Linear Regression and Future Revenue Prediction')
    plt.legend()
    plt.xticks(rotation=45)

    # Print predicted revenues for the end date
    end_date_prediction = model.predict([[ (pd.to_datetime(end_date) - data['Date'].min()).days ]])
    print(f'Predicted Revenue for {end_date}: {end_date_prediction[0]:.2f} USD')

    plt.show()

    # Calculate performance metrics on the training set
    y_pred_train = model.predict(X_train)
    mae_train = mean_absolute_error(y_train, y_pred_train)
    mse_train = mean_squared_error(y_train, y_pred_train)
    rmse_train = sqrt(mse_train)
    r2_train = r2_score(y_train, y_pred_train)

    # Print the results
    print(f'Mean Absolute Error (MAE) on training data: {mae_train:.2f}')
    print(f'Mean Squared Error (MSE) on training data: {mse_train:.2f}')
    print(f'Root Mean Squared Error (RMSE) on training data: {rmse_train:.2f}')
    print(f'R-squared (R²) on training data: {r2_train:.4f}')

# Example usage
linear_regression_and_metrics(abb_data, end_date, test_size=0.2, random_state=42)

Let's now apply the same code, but for the dataset starting from 2015.

In [ ]:
# Select only data related to Revenue, Electrification and starting from 2015
abb_data_new = abb_data[abb_data['Date'] > '2015-01-01']

# Use previously defined formula
linear_regression_and_metrics(abb_data_new, end_date, test_size=0.2, random_state=42)

In summary, Model 1 outperforms Model 2 on the training set based on various metrics. Model 1 exhibits lower Mean Absolute Error (MAE), Mean Squared Error (MSE), and Root Mean Squared Error (RMSE), indicating better predictive accuracy. Additionally, Model 1 achieves a higher R-squared (R²) value, suggesting a stronger fit to the training data.

Now, let's try to create a proper dataset with new Values and Dates to see which macroeconomic inditaor might be related with revenue evolution.

In [ ]:
def svm_regression_model(data, end_date):
    # Support Vector Machine for regression model
    X = (data['Date'] - data['Date'].min()).dt.days.values.reshape(-1, 1)
    y = data['Value']

    # Train a Support Vector Machine for regression model
    svm_model = SVR(kernel='linear')  # You can choose other kernels like 'rbf', 'poly', etc.
    svm_model.fit(X, y)

    # Generate dates up to the specified end date
    future_dates = pd.date_range(start=data['Date'].min(), end=end_date, freq='M')
    future_days = (future_dates - data['Date'].min()).days.values.reshape(-1, 1)

    # Predict revenues for future dates
    future_predictions = svm_model.predict(future_days)

    # Create the plot
    plt.figure(figsize=(12, 6))
    plt.scatter(data['Date'], data['Value'], label='Actual Data', color='red')
    plt.plot(data['Date'], svm_model.predict(X), label='SVM Regression', color='blue')
    plt.plot(future_dates, future_predictions, label='Predicted Data', linestyle='dashed', color='green')
    plt.xlabel('Date')
    plt.ylabel('Revenue (USD)')
    plt.title('SVM Regression and Future Revenue Prediction')
    plt.legend()
    plt.xticks(rotation=45)

    # Print predicted revenues for the end date
    end_date_prediction = svm_model.predict([[ (pd.to_datetime(end_date) - data['Date'].min()).days ]])
    print(f'Predicted Revenue for {end_date}: {end_date_prediction[0]:.2f} USD')

    # Performance Metrics
    y_pred_train = svm_model.predict(X)
    mae_train = mean_absolute_error(y, y_pred_train)
    mse_train = mean_squared_error(y, y_pred_train)
    rmse_train = np.sqrt(mse_train)
    r2_train = r2_score(y, y_pred_train)

    print(f'Mean Absolute Error (MAE) on training data: {mae_train:.2f}')
    print(f'Mean Squared Error (MSE) on training data: {mse_train:.2f}')
    print(f'Root Mean Squared Error (RMSE) on training data: {rmse_train:.2f}')
    print(f'R-squared (R²) on training data: {r2_train:.4f}')

    plt.show()

# Call the function
svm_regression_model(abb_data, end_date)

In [ ]:
def xgboost_model(data, end_date):
    # XGBoost model
    X = (data['Date'] - data['Date'].min()).dt.days.values.reshape(-1, 1)
    y = data['Value']

    # Train an XGBoost model
    xgboost_model = xgb.XGBRegressor(objective ='reg:squarederror')  # You can adjust hyperparameters as needed
    xgboost_model.fit(X, y)

    # Generate dates up to the specified end date
    future_dates = pd.date_range(start=data['Date'].min(), end=end_date, freq='M')
    future_days = (future_dates - data['Date'].min()).days.values.reshape(-1, 1)

    # Predict revenues for future dates
    future_predictions = xgboost_model.predict(future_days)

    # Create the plot
    plt.figure(figsize=(12, 6))
    plt.scatter(data['Date'], data['Value'], label='Actual Data', color='red')
    plt.plot(data['Date'], xgboost_model.predict(X), label='XGBoost Regression', color='blue')
    plt.plot(future_dates, future_predictions, label='Predicted Data', linestyle='dashed', color='green')
    plt.xlabel('Date')
    plt.ylabel('Revenue (USD)')
    plt.title('XGBoost Regression and Future Revenue Prediction')
    plt.legend()
    plt.xticks(rotation=45)

    # Print predicted revenues for the end date
    end_date_prediction = xgboost_model.predict([[ (pd.to_datetime(end_date) - data['Date'].min()).days ]])
    print(f'Predicted Revenue for {end_date}: {end_date_prediction[0]:.2f} USD')

    # Performance Metrics
    y_pred_train = xgboost_model.predict(X)
    mae_train = mean_absolute_error(y, y_pred_train)
    mse_train = mean_squared_error(y, y_pred_train)
    rmse_train = np.sqrt(mse_train)
    r2_train = r2_score(y, y_pred_train)

    print(f'Mean Absolute Error (MAE) on training data: {mae_train:.2f}')
    print(f'Mean Squared Error (MSE) on training data: {mse_train:.2f}')
    print(f'Root Mean Squared Error (RMSE) on training data: {rmse_train:.2f}')
    print(f'R-squared (R²) on training data: {r2_train:.4f}')

    plt.show()

# Call the function
xgboost_model(abb_data, end_date)

In [ ]:
# Select only data related to Revenue and ABB Electrification
electrification_data = df[(df['CIQ Formula'] == 'Revenue') & (df['Company name'] == 'ABB') & (df['Segment'] == 'Electrification')]

# Select data for linear regression model
X = (electrification_data['Date'] - electrification_data['Date'].min()).dt.days.values.reshape(-1, 1)
y = electrification_data['Value']

# Linear regression model
model = LinearRegression()
model.fit(X, y)
future_dates = pd.date_range(start=electrification_data['Date'].min(), end='2028-12-31', freq='M')
future_days = (future_dates - electrification_data['Date'].min()).days.values.reshape(-1, 1)

# Predict revenues for future dates and create a DataFrame
future_predictions = model.predict(future_days)
predictions_df = pd.DataFrame({'Date': future_dates, 'Comapny name': 'ABB', 'Segment': 'Electrification', 'Measure': 'Revenue', 'Value': future_predictions})

predictions_df

In [ ]:
# Data type sanity-check
predictions_df.dtypes

# Macroeconomics file - Data modelling

In [ ]:
macro = pd.read_csv("WEOOct2023all.csv")
macro

In [ ]:
# Unpivot the DataFrame, delete all unnecessary columns and all NaN's. Leave dates starting from 2010
macro = macro.drop(['WEO Country Code', 'WEO Subject Code', 'Subject Notes', 'Country/Series-specific Notes'], axis=1)
macro = pd.melt(macro, id_vars=['ISO', 'Country', 'Subject Descriptor', 'Units', 'Scale'], var_name='Date', value_name='Value')
macro = macro.dropna(how='all')
macro = macro.dropna(subset=['Value'])
macro = macro[macro['Date'] != 'Estimates Start After']
macro = macro[macro['Date'] >= '2010']

macro

In [ ]:
# Data type sanity-check
print(macro.dtypes)

In [ ]:
# Fix formats and sort by ISO and Date
macro['Date'] = pd.to_datetime(macro['Date'], errors='coerce')
macro['Value'] = pd.to_numeric(macro['Value'], errors='coerce')
macro = macro.sort_values(by=['ISO', 'Date'])

macro

In [ ]:
# Investigate Units
unique_units = macro['Units'].unique()
unique_units

In [ ]:
# Remove unnecessary Units
values_to_remove = ['National currency', 'Percent change','Purchasing power parity; international dollars', 'Index',
       'Purchasing power parity; 2017 international dollar', 'Percent',
       'National currency per current international dollar',
       'Percent of GDP', 'Percent of total labor force', 'Percent of potential GDP']
macro = macro[~macro['Units'].isin(values_to_remove)]
macro = macro.dropna(subset=['Value'])

macro

In [ ]:
#Delete more unnecessary columns
macro = macro.drop(['ISO', 'Units', 'Scale'], axis=1)
macro

Now the dataset with macroeconomic indicators contains only convinient stuff. Next step is to pivot measures.

In [ ]:
# Pivot table and reset index
pivot_macro = macro.pivot(index=['Country', 'Date'], columns='Subject Descriptor', values='Value').reset_index()
pivot_macro

Column "Employment" seems to be empty - let's drop it too.

In [ ]:
# Delete 'Employment' column
pivot_macro = pivot_macro.drop(['Employment'], axis=1)
pivot_macro

This dataset is almost ready to be merged with the first one. There are only 2 additional columns missing. Since we are focuing on ABB Electrification this time, I will add these values as strings for the time being.

In [ ]:
# Add two columns
pivot_macro['Comapny name'] = 'ABB'
pivot_macro['Segment'] = 'Electrification'
pivot_macro

Let's take another look at database with ABB EL revenues to see if we can merge it with new Macro dataset

In [ ]:
predictions_df

Once we try to merge these two columns, there might be an issue with Date column. Macro database is showing full year data, while predictions df is quarterly based. Let's change this column to show full year in both cases.

In [ ]:
# Change 'Date' column to show year only
pivot_macro['Date'] = pivot_macro['Date'].dt.year
pivot_macro

In [ ]:
# Change 'Date' column to show year only and group values to show total for full year
predictions_df['Date'] = predictions_df['Date'].dt.year
annual_sum = predictions_df.groupby(['Comapny name', 'Segment', 'Measure', 'Date'])['Value'].sum().reset_index()
annual_sum = annual_sum.drop(['Measure'], axis=1)
annual_sum

Now both datasets are ready to consolidate.

In [ ]:
# Merge 'annual_sum' and 'pivot_macro'. Drop NaNs in 'Value' column
merged_df = pd.merge(annual_sum, pivot_macro, on=['Comapny name', 'Segment', 'Date'], how='outer')
merged_df = merged_df.dropna(subset=['Value'])
merged_df

# Applying Random Forest model to merged dataset

In [ ]:
# Split the data into features (X) and target variable (y)
X = merged_df[['Date', 'Current account balance', 'Gross domestic product per capita, current prices', 'Gross domestic product, current prices', 'Population']]
y = merged_df['Value']

# Impute missing values using the mean strategy
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)

# Create and train a Random Forest model
rf_model = RandomForestRegressor()
rf_model.fit(X_imputed, y)
results_df = pd.DataFrame(columns=['Country', 'Mean Squared Error', 'Current account balance Importance Check', 'GDP per capita Importance Check', 'GDP Importance Check', 'Population Importance Check', 'Population Value'])

# Assign feature importance values for each country
for country in merged_df['Country'].unique():
    country_index = merged_df['Country'] == country
    country_mse = mean_squared_error(y[country_index], rf_model.predict(X_imputed[country_index]))
    
    # Train a new model for each country to get feature importances specific to that country
    country_rf_model = RandomForestRegressor()
    country_rf_model.fit(X_imputed[country_index], y[country_index])
    
    country_result = {
        'Country': country,
        'Mean Squared Error': country_mse,
        'Current account balance Importance Check': country_rf_model.feature_importances_[0],
        'GDP per capita Importance Check': country_rf_model.feature_importances_[1],
        'GDP Importance Check': country_rf_model.feature_importances_[2],
        'Population Importance Check': country_rf_model.feature_importances_[3],
        'Population Value': X.loc[country_index, 'Population'].iloc[0]
    }
    
    results_df = pd.concat([results_df, pd.DataFrame([country_result])], ignore_index=True)

# Sort the results by Population Value in descending order
results_df = results_df.sort_values(by='Population Value', ascending=False)

results_df